In [17]:
"""
condo_scoring_v2.py

- มี 3 mode คือ "BTS", "Vehicle", "Mixed" จะแบ่งตามการเดินทางของผู้ใช้
- แต่ละโหมดจะมีการคำนวณ score ที่แตกต่างกัน โดยถ้าใช้ BTS จะตัดปัญหาการจราจรออกไป
- คำนวณความคุ้มค่าของคอนโดโดยใช้ปัจจัยต่างๆ เช่น จำนวนปัญหาในพื้นที่, ราคาเฉลี่ยต่อตารางเมตร เป็นต้น
- สามารถปรับระยะทางหรือรัศมีในการนับปัญหาโดยรอบได้
- แต่ละปัญหาจะมีการถ่วงน้ำหนักปัจจัยต่างๆ แตกต่างกันไป เช่น น้ำท่วมมีผลกระทบมากกว่าปัญหาขยะ
- แต่ละ ticket จะมีผลไม่เท่ากันขึ้นอยู่กับสถานะของ ticket นั้นๆ เช่นถ้า ticker ปิดแล้วจะมีผลกระทบน้อยกว่า ticket ที่ยังเปิดอยู่ และ ticket ที่มีการ reopen จะมีผลกระทบมากกว่า ticket ที่ไม่มีการ reopen
- มี column ชื่อ quality_label คือการบอกความคุ้มค่าของคอนโดต่อจำนวนปัญหา
- โดยผลลัพธ์จะมี 2 แบบคือ แบบแยกตามแต่ละเขต (detailed) และ แบบสรุปรวมทุกเขต (summary)
"""

import os
import re
from typing import Dict, Any
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import pandas as pd
import geopandas as gpd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


In [18]:
# ===================== CONFIG =====================

TARGET_DISTRICTS_THAI = [
    "จตุจักร",
    "ประเวศ",
    "วัฒนา",
    "บางกะปิ",
    "คลองเตย",
    "บางแค",
    "ปทุมวัน",
    "บางเขน",
]

INPUT_DIR = "../Data/clean"
OUTPUT_DIR = "../Data/score_v2"

CONDO_FILES = [
    f"{INPUT_DIR}/condo_จตุจักร_clean.csv",
    f"{INPUT_DIR}/condo_ประเวศ_clean.csv",
    f"{INPUT_DIR}/condo_วัฒนา_clean.csv",
    f"{INPUT_DIR}/condo_บางกะปิ_clean.csv",
    f"{INPUT_DIR}/condo_คลองเตย_clean.csv",
    f"{INPUT_DIR}/condo_บางแค_clean.csv",
    f"{INPUT_DIR}/condo_ปทุมวัน_clean.csv",
    f"{INPUT_DIR}/condo_บางเขน_clean.csv",
]

CONDO_LAT_COL = "latitude"
CONDO_LON_COL = "longitude"
CONDO_DISTRICT_COL = "district_name"
CONDO_NAME_COL = "condo_name"

CONDO_PRICE_COL = "Price"
CONDO_SIZE_COL = "Room_Size"

TRAFFY_FILE = f"{INPUT_DIR}/bangkok_traffy_clean.csv"

BTS_FILE = f"{INPUT_DIR}/bts_station.csv"
BTS_LAT_COL = "lat"
BTS_LON_COL = "lng"
BTS_NAME_COL = "name"

CONDO_BUFFER_RADIUS_M = 1000 # ระยะรัศมีในการนับปัญหาโดยรอบคอนโด (เมตร)
BTS_NEAR_RADIUS_M = 500 # ระยะห่างจาก BTS เพื่อพิจารณาว่าใกล้ BTS หรือไม่ (เมตร)
USER_TRAVEL_MODE = "bts" # "bts", "vehicle", "mixed"


TYPE_TO_CATEGORY: Dict[str, str] = {
    # ---- ROAD / TRAVEL ----
    "ถนน": "road",
    "ทางเท้า": "road",
    "จราจร": "road",
    "การเดินทาง": "road",
    "สะพาน": "road",
    "ป้ายจราจร": "road",
    "ป้าย": "road",

    # ---- FLOOD / DRAINAGE ----
    "น้ำท่วม": "flood",


    # ---- SAFETY / SECURITY ----
    "ความปลอดภัย": "safety",
    "แสงสว่าง": "safety",
    "สายไฟ": "safety",
    "สัตว์จรจัด": "safety",
    "คนจรจัด": "safety",

    # ---- NOISE ----
    "เสียงรบกวน": "noise",

    # ---- CLEANLINESS / OBSTRUCTION ----
    "ความสะอาด": "cleanliness",
    "กีดขวาง": "cleanliness",
    "ห้องน้ำ": "cleanliness",

    # ---- ENVIRONMENT / AIR ----
    "ต้นไม้": "environment",
    "PM2.5": "environment",
    "ท่อระบายน้ำ": "flood",
    "คลอง": "flood",

    # ---- META / OTHER ----
    "ร้องเรียน": "other",
    "สอบถาม": "other",
    "เสนอแนะ": "other",
    "อื่นๆ": "other",
}

DEFAULT_CATEGORY = "other"

# 10) น้ำหนักของแต่ละ category (ยิ่งสูง = ยิ่งร้ายแรง)
CATEGORY_WEIGHTS: Dict[str, float] = {
    "flood": 4.0,
    "safety": 3.5,
    "noise": 3.0,
    "road": 2.5,
    "cleanliness": 2.5,
    "environment": 3.0,
    "other": 1.0,
}

RESOLVED_STATES = ["DONE", "Done", "done", "เสร็จสิ้น"]
REOPEN_PENALTY_PER_TIME = 0.3
REOPEN_PENALTY_MAX_TIMES = 3

In [ ]:
# ===================== UTIL =====================

def clean_district(name: Any) -> str | None:
    if pd.isna(name):
        return None
    s = str(name).strip()
    if s.startswith("เขต"):
        s = s[3:].strip()
    return s


def split_type_list(raw: Any) -> list[str]:
    if pd.isna(raw):
        return []
    s = str(raw).strip()
    if s.startswith("(") and s.endswith(")"):
        s = s[1:-1]
    if not s:
        return []
    parts = [p.strip() for p in s.split(",") if p.strip()]
    return parts


def map_type_to_category(t: Any) -> str:
    if pd.isna(t):
        return DEFAULT_CATEGORY
    s = str(t).strip()
    return TYPE_TO_CATEGORY.get(s, DEFAULT_CATEGORY)


def compute_resolution_factor(state: Any, count_reopen: Any) -> float:
    state_str = str(state).strip()
    try:
        reopen = int(count_reopen)
    except (ValueError, TypeError):
        reopen = 0
    base = 1.0 if state_str in RESOLVED_STATES else 1.5
    penalty = REOPEN_PENALTY_PER_TIME * min(reopen, REOPEN_PENALTY_MAX_TIMES)
    return base + penalty


def get_travel_factor(mode: str, near_bts: bool) -> float:
    mode = mode.lower()
    if mode == "bts":
        return 0.0 if near_bts else 1.0
    if mode == "mixed":
        return 0.5 if near_bts else 1.0
    return 1.0


def classify_price_risk(row, price_th, risk_th) -> str:
    price_score = row["price_score_0_100"]
    risk_score = row["risk_score_0_100"]
    expensive = price_score > price_th
    cheap = price_score < price_th
    low_risk = risk_score < risk_th
    high_risk = risk_score > risk_th
    if not (expensive or cheap):
        if low_risk:
            return "Good"
        elif high_risk:
            return "Okay"
        else:
            return "Okay"
    if expensive and low_risk:
        return "Perfect"
    if expensive and not low_risk:
        return "Bad"
    if cheap and low_risk:
        return "Good"
    return "Okay"


def parse_price(x: Any) -> float | np.ndarray:
    if pd.isna(x):
        return np.nan
    s = str(x)
    s = re.sub(r"[^0-9.,]", "", s)
    s = s.replace(",", "")
    if s == "":
        return np.nan
    try:
        return float(s)
    except ValueError:
        return np.nan


def parse_room_size(x: Any) -> float | np.ndarray:
    if pd.isna(x):
        return np.nan
    s = str(x)
    m = re.search(r"([0-9]+(\.[0-9]+)?)", s)
    if not m:
        return np.nan
    try:
        return float(m.group(1))
    except ValueError:
        return np.nan

# -------------------------- Clustering Part --------------------------

def haversine_vectorized(lat1, lon1, lat2, lon2):
    """
    คำนวณระยะทาง (km) ระหว่างจุด 1 จุด (lat1) กับ หลายจุด (lat2 array)
    """
    R = 6371  # รัศมีโลก (km)
    
    # แปลง Degree -> Radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    return R * c

def sum_problem_by_type_vectorized(condo_lat, condo_lon, traffy_df):

    RADIUS_KM = CONDO_BUFFER_RADIUS_M / 1000.0
    BASE_SCORE = 1.0
    
    # ⚖️ Weight (น้ำหนักการหักคะแนน)
    W_UNFINISHED = 0.05       # งานไม่เสร็จ (หักหนัก)
    W_FINISHED_BAD = 0.01     # งานเสร็จแต่ดาวน้อย (<3 ดาว)
    W_FINISHED_GOOD = 0.001   # งานเสร็จและดี (หักนิดเดียว เป็นแค่ Noise)

    if traffy_df.empty:
        return BASE_SCORE

    # 1. คำนวณระยะทางทั้งหมดรวดเดียว
    dists = haversine_vectorized(condo_lat, condo_lon, traffy_df['lat'].values, traffy_df['lon'].values)
    
    # 2. กรองเฉพาะที่อยู่ในรัศมี (Filter Mask)
    mask = dists <= RADIUS_KM
    nearby_issues = traffy_df[mask].copy()
    nearby_dists = dists[mask]
    
    if len(nearby_issues) == 0:
        return BASE_SCORE

    # 3. เตรียม Weight เบื้องต้นตาม State
    # สร้าง array ของ weight ขึ้นมา
    weights = np.where(
        nearby_issues['state'] == 'เสร็จสิ้น',
        np.where(nearby_issues['star'].fillna(0) < 3, W_FINISHED_BAD, W_FINISHED_GOOD),
        W_UNFINISHED
    )
    
    # 4. Keyword Boosting: map each nearby issue type to a category (using TYPE_TO_CATEGORY)
    # และนำ weight ของ category มาคูณกับ weight เดิม (vectorized)
    types_series = nearby_issues['type'].astype(str)
    # start with default category for all rows
    categories = np.array([DEFAULT_CATEGORY] * len(types_series))
    for keyword, cat in TYPE_TO_CATEGORY.items():
        # escape keyword (handles "PM2.5" etc.) and build mask
        mask = types_series.str.contains(re.escape(keyword), na=False).values
        categories[mask] = cat
    # map categories to multipliers and apply
    multipliers = np.array([CATEGORY_WEIGHTS.get(c, CATEGORY_WEIGHTS[DEFAULT_CATEGORY]) for c in categories])
    weights = weights * multipliers
    
    # 5. Distance Decay (ยิ่งไกล ยิ่งผลน้อย)
    # ที่ระยะ 0km -> factor = 1.0
    # ที่ระยะ RADIUS_KM -> factor = 0.0
    dist_factors = 1.0 - (nearby_dists / RADIUS_KM)
    
    # 6. รวม Penalty
    # Penalty = sum(weight * dist_factor)
    total_penalty = np.sum(weights * dist_factors)
    
    return total_penalty

def kmeans_clustering(df, k = 4):
    df_pca = df.copy().reset_index(drop=True)

    df_pca['Room_Size_sqm'] = df_pca['Room_Size'].str.replace(' ตร.ม.', '', regex=False).astype(float, errors='ignore')

    df_pca['Price_Per_SqM'] = df_pca['Price'] / df_pca['Room_Size_sqm']
    df_pca['Affordability_Index'] = -df_pca['Price_Per_SqM']

    features = [
        'Price',
        'Affordability_Index',
        'latitude',
        'longitude',
        'weighted_problem_count'
    ]

    X = df_pca[features].copy()
    X['positive_influence_problem'] = -X['weighted_problem_count']
    scoring_features = [
        'Price',
        'Affordability_Index',
        'latitude',
        'longitude',
        'positive_influence_problem'
    ]

    original_index = X.index
    X_processed = X[scoring_features].dropna()
    processed_indices = X_processed.index 

    scaler = StandardScaler()
    X_scaled_array = scaler.fit_transform(X_processed)

    # Apply PCA
    pca = PCA(n_components=3)
    principal_components = pca.fit_transform(X_scaled_array)

    variance_ratios = pca.explained_variance_ratio_
    pc1 = principal_components[:, 0]
    pc2 = principal_components[:, 1]
    pc3 = principal_components[:, 2]

    w1, w2, w3 = variance_ratios[0], variance_ratios[1], variance_ratios[2]
    total_weight = w1 + w2 + w3

    weighted_score_series = ( (w1 * pc1) + (w2 * pc2) + (w3 * pc3) ) / total_weight

    full_score_series = pd.Series(np.nan, index=df_pca.index)
    full_score_series.loc[processed_indices] = weighted_score_series

    df_pca['Weighted_Composite_Score'] = full_score_series

    X_clusters = principal_components

    kmeans_model = KMeans(n_clusters=k, random_state=42, n_init=10)
    cluster_labels = kmeans_model.fit_predict(X_clusters)

    cluster_df = pd.DataFrame(
        cluster_labels,
        index=processed_indices,
        columns=['Cluster_Label']
    )

    df_kmean = df_pca.join(cluster_df, how='left')

    df_kmean['Cluster_Label'] = df_kmean['Cluster_Label'].astype('category')

    return df_kmean


In [23]:
# ===================== MAIN =====================

def main():
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # LOAD condos
    condo_list = []
    for f in CONDO_FILES:
        df = pd.read_csv(f)
        condo_list.append(df)
    condos = pd.concat(condo_list, ignore_index=True)

    condos["district_clean"] = condos[CONDO_DISTRICT_COL].apply(clean_district)
    condos = condos[condos["district_clean"].isin(TARGET_DISTRICTS_THAI)].copy()
    condos = condos.dropna(subset=[CONDO_LAT_COL, CONDO_LON_COL])
    condos["condo_id"] = range(len(condos))

    # LOAD traffy
    traffy = pd.read_csv(TRAFFY_FILE)
    traffy["district_clean"] = traffy["district"].apply(clean_district)
    traffy = traffy[traffy["district_clean"].isin(TARGET_DISTRICTS_THAI)].copy()
    traffy = traffy.dropna(subset=["lat", "lon"])

    traffy["type_list"] = traffy["type"].apply(split_type_list)
    traffy_long = traffy.explode("type_list").rename(columns={"type_list": "type_clean"})
    traffy_long["type_clean"] = traffy_long["type_clean"].fillna("อื่นๆ")
    traffy_long["category"] = traffy_long["type_clean"].apply(map_type_to_category)

    problem_sum_pca = condos.apply(
        lambda row: sum_problem_by_type_vectorized(row['latitude'], row['longitude'], traffy), 
        axis=1
    )
    condos_pca = condos.copy()
    condos_pca["weighted_problem_count"] = problem_sum_pca
    condos_kmean = kmeans_clustering(condos_pca)

    # Geo
    condos_gdf = gpd.GeoDataFrame(
        condos,
        geometry=gpd.points_from_xy(condos[CONDO_LON_COL], condos[CONDO_LAT_COL]),
        crs="EPSG:4326",
    )
    traffy_gdf = gpd.GeoDataFrame(
        traffy_long,
        geometry=gpd.points_from_xy(traffy_long["lon"], traffy_long["lat"]),
        crs="EPSG:4326",
    )
    condos_gdf = condos_gdf.to_crs(epsg=3857)
    traffy_gdf = traffy_gdf.to_crs(epsg=3857)

    # buffer & join
    condo_buffer = condos_gdf.copy()
    condo_buffer["geometry"] = condo_buffer.geometry.buffer(CONDO_BUFFER_RADIUS_M)
    condo_area = condo_buffer[["condo_id", CONDO_NAME_COL, "district_clean", "geometry"]].copy()

    joined = gpd.sjoin(
        traffy_gdf,
        condo_area,
        predicate="within",
        how="inner",
    )

    # ticket scoring
    joined["category_weight"] = joined["category"].map(CATEGORY_WEIGHTS).fillna(CATEGORY_WEIGHTS.get(DEFAULT_CATEGORY, 1.0))
    joined["resolution_factor"] = joined.apply(lambda row: compute_resolution_factor(row["state"], row["count_reopen"]), axis=1)
    joined["ticket_score"] = joined["category_weight"] * joined["resolution_factor"]
    joined["is_unresolved"] = ~joined["state"].isin(RESOLVED_STATES)

    # unique ticket counts (per condo) — ไม่ใช่หลัง explode
    unique_tickets = joined.groupby("condo_id")["ticket_id"].nunique().reset_index().rename(columns={"ticket_id": "unique_tickets"})
    unique_unresolved = joined[joined["is_unresolved"] == True].groupby("condo_id")["ticket_id"].nunique().reset_index().rename(columns={"ticket_id": "unique_unresolved"})
    condo_uniques = unique_tickets.merge(unique_unresolved, on="condo_id", how="left")

    # aggregate per condo & category (after explode)
    per_condo_cat = joined.groupby(["condo_id", "category"], as_index=False).agg(
        problems_count=("ticket_id", "count"),
        total_score=("ticket_score", "sum"),
        unresolved_count=("is_unresolved", "sum"),
    )

    condo_cat_counts = per_condo_cat.pivot(index="condo_id", columns="category", values="problems_count").fillna(0)
    condo_cat_counts.columns = [f"count_{c}" for c in condo_cat_counts.columns]
    condo_cat_counts = condo_cat_counts.reset_index()

    condo_cat_scores = per_condo_cat.pivot(index="condo_id", columns="category", values="total_score").fillna(0)
    condo_cat_scores.columns = [f"score_{c}" for c in condo_cat_scores.columns]
    condo_cat_scores = condo_cat_scores.reset_index()

    condo_cat_unresolved = per_condo_cat.pivot(index="condo_id", columns="category", values="unresolved_count").fillna(0)
    condo_cat_unresolved.columns = [f"unresolved_{c}" for c in condo_cat_unresolved.columns]
    condo_cat_unresolved = condo_cat_unresolved.reset_index()

    condo_totals = per_condo_cat.groupby("condo_id", as_index=False).agg(
        total_problems=("problems_count", "sum"),
        total_score=("total_score", "sum"),
        total_unresolved=("unresolved_count", "sum"),
    )

    # merge features
    condos_features = condos_gdf.merge(condo_totals, on="condo_id", how="left")
    condos_features = condos_features.merge(condo_uniques, on="condo_id", how="left")
    condos_features = condos_features.merge(condo_cat_counts, on="condo_id", how="left")
    condos_features = condos_features.merge(condo_cat_scores, on="condo_id", how="left")
    condos_features = condos_features.merge(condo_cat_unresolved, on="condo_id", how="left")

    feature_cols = [c for c in condos_features.columns if c.startswith("count_") or c.startswith("score_") or c.startswith("unresolved_") or c in ["total_problems", "total_score", "total_unresolved", "unique_tickets", "unique_unresolved"]]
    condos_features[feature_cols] = condos_features[feature_cols].fillna(0)

    # BTS nearest
    bts = pd.read_csv(BTS_FILE)
    bts_gdf = gpd.GeoDataFrame(bts, geometry=gpd.points_from_xy(bts[BTS_LON_COL], bts[BTS_LAT_COL]), crs="EPSG:4326").to_crs(epsg=3857)

    nearest = gpd.sjoin_nearest(condos_features, bts_gdf[[BTS_NAME_COL, "geometry"]], how="left", distance_col="dist_to_bts_m")
    condos_features["dist_to_bts_m"] = nearest["dist_to_bts_m"]
    condos_features["nearest_bts"] = nearest[BTS_NAME_COL]
    condos_features["near_bts"] = condos_features["dist_to_bts_m"] <= BTS_NEAR_RADIUS_M

    # compute risk (normalize per category -> weight)
    category_to_count_col = {}
    for cat in CATEGORY_WEIGHTS.keys() | {DEFAULT_CATEGORY}:
        col_name = f"count_{cat}"
        if col_name in condos_features.columns:
            category_to_count_col[cat] = col_name

    risk_components_cols = []
    for cat, count_col in category_to_count_col.items():
        max_val = condos_features[count_col].max()
        norm_col = f"norm_{cat}"
        if max_val > 0:
            condos_features[norm_col] = condos_features[count_col] / max_val
        else:
            condos_features[norm_col] = 0.0
        w = CATEGORY_WEIGHTS.get(cat, CATEGORY_WEIGHTS.get(DEFAULT_CATEGORY, 1.0))
        risk_col = f"risk_{cat}"
        condos_features[risk_col] = condos_features[norm_col] * w
        risk_components_cols.append(risk_col)

    road_risk_col = "risk_road"
    non_travel_risk_cols = [c for c in risk_components_cols if c != road_risk_col]
    condos_features["risk_non_travel"] = condos_features[non_travel_risk_cols].sum(axis=1) if non_travel_risk_cols else 0.0
    condos_features["risk_travel_raw"] = condos_features[road_risk_col] if road_risk_col in condos_features.columns else 0.0
    condos_features["travel_factor"] = condos_features["near_bts"].apply(lambda nb: get_travel_factor(USER_TRAVEL_MODE, bool(nb)))
    condos_features["risk_travel_effective"] = condos_features["risk_travel_raw"] * condos_features["travel_factor"]
    condos_features["raw_risk"] = condos_features["risk_non_travel"] + condos_features["risk_travel_effective"]

    raw_min = condos_features["raw_risk"].min()
    raw_max = condos_features["raw_risk"].max()
    condos_features["risk_score_0_100"] = ((condos_features["raw_risk"] - raw_min) / (raw_max - raw_min) * 100) if raw_max > raw_min else 0.0
    condos_features["safety_score_0_100"] = 100 - condos_features["risk_score_0_100"]

    # price parsing & compute price per sqm
    condos_features[CONDO_PRICE_COL] = condos_features[CONDO_PRICE_COL].apply(parse_price)
    condos_features[CONDO_SIZE_COL] = condos_features[CONDO_SIZE_COL].apply(parse_room_size)
    condos_features["price_per_sqm"] = condos_features[CONDO_PRICE_COL] / condos_features[CONDO_SIZE_COL]
    condos_features.loc[(condos_features["price_per_sqm"] <= 0) | (~np.isfinite(condos_features["price_per_sqm"])), "price_per_sqm"] = np.nan

    min_p = condos_features["price_per_sqm"].min()
    max_p = condos_features["price_per_sqm"].max()
    if pd.notna(min_p) and pd.notna(max_p) and max_p > min_p:
        condos_features["price_score_0_100"] = (condos_features["price_per_sqm"] - min_p) / (max_p - min_p) * 100
    else:
        condos_features["price_score_0_100"] = 50.0
    condos_features["price_score_0_100"] = condos_features["price_score_0_100"].fillna(50.0)

    price_th = condos_features["price_score_0_100"].median()
    risk_th = condos_features["risk_score_0_100"].median()
    condos_features["quality_label"] = condos_features.apply(lambda row: classify_price_risk(row, price_th, risk_th), axis=1)
    condos_features["value_index"] = (100 - condos_features["risk_score_0_100"]) - condos_features["price_score_0_100"]

    # overall score
    affordability = 100 - condos_features["price_score_0_100"]
    condos_features["overall_score_0_100"] = 0.7 * condos_features["safety_score_0_100"] + 0.3 * affordability

    # ========== NEW: percentiles IN DISTRICT ==========
    # For each district, compute percentile rank (0-100) of certain scores:
    pct_cols = {
        "overall_score_0_100": "overall_pct_in_district",
        "safety_score_0_100": "safety_pct_in_district",
        "price_score_0_100": "price_pct_in_district",
        "value_index": "value_index_pct_in_district",
    }

    # Use groupby + rank(pct=True) to compute percentile within district
    for score_col, out_col in pct_cols.items():
        # default 0 if district has single item or NaN
        condos_features[out_col] = condos_features.groupby("district_clean")[score_col].transform(
            lambda x: x.rank(method="average", pct=True) * 100 if x.notna().sum() > 0 else 0
        )
        # fill NaN with 0
        condos_features[out_col] = condos_features[out_col].fillna(0)

    # ========== SAVE overall + per-district files ==========

    os.makedirs(OUTPUT_DIR, exist_ok=True)
    output_df = condos_features.drop(columns="geometry")

    cluster_map = {
        0: "Premium Low-Risk",
        1: "Value & High-Risk",
        2: "Ultra-Luxury",
        3: "Affordable Entry-Level",
    }
    labels_col = "Cluster_Label"
    # convert categorical to object first, then map and fill missing
    mapped = condos_kmean[labels_col].astype(object).map(cluster_map)
    output_df["Cluster_Label"] = mapped.fillna("Other")

    # --- Reorder columns: put overall_pct_in_district then overall_score_0_100 at the far right ---
    # drop geometry if present
    if "geometry" in output_df.columns:
        output_df = output_df.drop(columns=["geometry"])

    # now save the overall file
    # final_columns = [
    #     'Project_Name', 'condo_name', 'Price', 'price_per_sqm',
    #     'Room_Type', 'Floor', 'Bedrooms', 'Bathrooms', 'Room_Size',
    #     'Original_Link', 'district_clean', 'latitude', 'longitude',
    #     'price_score_0_100', 'quality_label', 'overall_pct_in_district',
    #     'overall_score_0_100']
    original_columns = ['Project_Name', 'condo_name', 'Price', 'Room_Type', 'Floor', 'Bedrooms', 
                        'Bathrooms', 'Room_Size', 'Original_Link', 'district_name', 'latitude', 
                        'longitude', 'district_clean', 'condo_id', 'total_problems', 'total_score', 
                        'total_unresolved', 'unique_tickets', 'unique_unresolved', 'count_cleanliness',
                         'count_environment', 'count_flood', 'count_noise', 'count_other', 'count_road', 
                         'count_safety', 'score_cleanliness', 'score_environment', 'score_flood', 'score_noise', 
                         'score_other', 'score_road', 'score_safety', 'unresolved_cleanliness', 'unresolved_environment',
                        'unresolved_flood', 'unresolved_noise', 'unresolved_other', 'unresolved_road', 'unresolved_safety',
                          'dist_to_bts_m', 'nearest_bts', 'near_bts', 'norm_safety', 'risk_safety', 'norm_environment', 'risk_environment',
                            'norm_other', 'risk_other', 'norm_road', 'risk_road', 'norm_cleanliness', 'risk_cleanliness', 'norm_flood', 
                            'risk_flood', 'norm_noise', 'risk_noise', 'risk_non_travel', 'risk_travel_raw', 'travel_factor', 
                            'risk_travel_effective', 'raw_risk', 'risk_score_0_100', 'safety_score_0_100', 'price_per_sqm', 
                            'price_score_0_100', 'quality_label', 'value_index', 'safety_pct_in_district', 'price_pct_in_district',
                              'value_index_pct_in_district', 'overall_pct_in_district', 'overall_score_0_100', 'Cluster_Label']
    output_df = output_df[original_columns]
    output_df['livability'] = output_df['overall_score_0_100']
    output_all = f"{OUTPUT_DIR}/condos_scored_all.csv"
    output_df.to_csv(output_all, index=False)
    print(f"Saved overall file (with reordered columns): {output_all}")


    # save per-district files with same column order
    districts = output_df["district_clean"].fillna("unknown").unique()
    for d in districts:
        if pd.isna(d):
            continue
        df_d = output_df[output_df["district_clean"] == d].copy()
        fname = f"{OUTPUT_DIR}/{d}_condos_scored.csv"
        df_d = df_d[original_columns]
        df_d.to_csv(fname, index=False)
        print("Saved per-district file:", fname)

    # save per-district files
    districts = condos_features["district_clean"].fillna("unknown").unique()
    saved_files = []
    for d in districts:
        if pd.isna(d):
            continue
        df_d = output_df[output_df["district_clean"] == d].copy()
        fname = f"{OUTPUT_DIR}/{d}_condos_scored.csv"
        df_d = df_d[original_columns]
        df_d.to_csv(fname, index=False)
        saved_files.append(fname)

    print("Saved per-district files:")
    for p in saved_files:
        print(" -", p)

    # sample head
    print(output_df[original_columns].head(15).to_string(index=False))


if __name__ == "__main__":
    main()


,Project_Name,condo_name,Price,Room_Type,Floor,Bedrooms,Bathrooms,Room_Size,Original_Link,district_name,latitude,longitude,district_clean,condo_id,weighted_problem_count,Room_Size_sqm,Price_Per_SqM,Affordability_Index,Weighted_Composite_Score,Cluster_Label
0,ขายคอนโด เดอะ เซนต์ เรสิเดนเซส อาคาร C ชั้น 29...,The Saint Residences,6790000.0,2 ห้องนอน,29,2 ห้องนอน,2 ห้องน้ำ,57 ตร.ม.,https://propertyhub.in.th/listings/%E0%B8%82%E...,เขตจตุจักร,13.811510,100.561078,จตุจักร,0,89.398338,57.0,119122.807018,-119122.807018,0.213985,2.0
1,ขายคอนโด เดอะ ไลน์ จตุจักร-หมอชิต อาคาร 1 ชั้น...,THE LINE Jatujak - Mochit,7900000.0,1 ห้องนอน,24,1 ห้องนอน,1 ห้องน้ำ,40 ตร.ม.,https://propertyhub.in.th/listings/%E0%B8%82%E...,เขตจตุจักร,13.805826,100.556463,จตุจักร,1,53.976252,40.0,197500.000000,-197500.000000,0.134146,2.0
2,ขายคอนโด เอ็ม จตุจักร อาคาร B ชั้น 31 2 ห้องนอ...,M Jatujak,7500000.0,2 ห้องนอน,31,2 ห้องนอน,2 ห้องน้ำ,56 ตร.ม.,https://propertyhub.in.th/listings/%E0%B8%82%E...,เขตจตุจักร,13.797088,100.551662,จตุจักร,2,51.756751,56.0,133928.571429,-133928.571429,0.263212,2.0
3,📌 ขายคอนโด เอ็ม จตุจักร อาคาร A ชั้น 8 1 ห้องน...,M Jatujak,3920000.0,1 ห้องนอน,8,1 ห้องนอน,1 ห้องน้ำ,28 ตร.ม.,https://propertyhub.in.th/listings/%E0%B8%82%E...,เขตจตุจักร,13.797088,100.551662,จตุจักร,3,51.756751,28.0,140000.000000,-140000.000000,0.297845,2.0
4,📌 ขายคอนโด เอ็ม จตุจักร อาคาร B ชั้น 14 1 ห้อง...,M Jatujak,5260000.0,1 ห้องนอน,14,1 ห้องนอน,1 ห้องน้ำ,33 ตร.ม.,https://propertyhub.in.th/listings/%E0%B8%82%E...,เขตจตุจักร,13.797088,100.551662,จตุจักร,4,51.756751,33.0,159393.939394,-159393.939394,0.222266,2.0


Saved overall file (with reordered columns): ../Data/score_v2/condos_scored_all.csv
Saved per-district file: ../Data/score_v2/จตุจักร_condos_scored.csv
Saved per-district file: ../Data/score_v2/ประเวศ_condos_scored.csv
Saved per-district file: ../Data/score_v2/วัฒนา_condos_scored.csv
Saved per-district file: ../Data/score_v2/บางกะปิ_condos_scored.csv
Saved per-district file: ../Data/score_v2/คลองเตย_condos_scored.csv
Saved per-district file: ../Data/score_v2/บางแค_condos_scored.csv
Saved per-district file: ../Data/score_v2/ปทุมวัน_condos_scored.csv
Saved per-district file: ../Data/score_v2/บางเขน_condos_scored.csv
Saved per-district files:
 - ../Data/score_v2/จตุจักร_condos_scored.csv
 - ../Data/score_v2/ประเวศ_condos_scored.csv
 - ../Data/score_v2/วัฒนา_condos_scored.csv
 - ../Data/score_v2/บางกะปิ_condos_scored.csv
 - ../Data/score_v2/คลองเตย_condos_scored.csv
 - ../Data/score_v2/บางแค_condos_scored.csv
 - ../Data/score_v2/ปทุมวัน_condos_scored.csv
 - ../Data/score_v2/บางเขน_condos_s